In [1]:
import requests
import json
import math
import pandas as pd
import hashlib
import sqlite3

In [2]:
token='4Hc4ussRHULctgSsgU9eDFntrruEyhiVzrb84RgxKnMz'
header={'Authorization': f'Bearer {token}'}

In [3]:
formList='https://api.typeform.com/forms?page_size=200&page={page}'
formItems='https://api.typeform.com/forms/{id}'
respList='https://api.typeform.com/forms/{id}/responses?page_size={psize}&page={page}'

In [4]:
response=requests.get(formList.format(page=1),
                      headers=header).json()

forms=[]

for f in response['items']:
    form={}
    form['id']=f['id']
    form['url']=f['_links']['display']
    form['title']=f['title']
    
    forms.append(form)

f=pd.DataFrame(columns=forms[0].keys())
forms=f.append(forms)
del f
forms.set_index('id',inplace=True)
forms.head()

,url,title
id,,
sCBW64,https://projetosaf.typeform.com/to/sCBW64,PDCA - Tela Inicial
deP0yD,https://projetosaf.typeform.com/to/deP0yD,PDCA - Gestão de Negócios
dlLAKk,https://projetosaf.typeform.com/to/dlLAKk,PDCA - Digitalize
HtP0WQ,https://projetosaf.typeform.com/to/HtP0WQ,PDCA - Vibra
plLZoV,https://projetosaf.typeform.com/to/plLZoV,PDCA - SAF (Auto)


In [5]:
forms.index

Index(['sCBW64', 'deP0yD', 'dlLAKk', 'HtP0WQ', 'plLZoV', 'v5epz2', 'tPX3NX',
       'pzDoa7', 'APiACy', 'WwuFx0', 'YhT1vU', 'tej3df', 'HM4z4y', 'kVGmcU',
       'fwCpgm', 'nQBceF', 'NspxTx', 'f38Erg', 'BeN2zg', 'VZnpZy', 'zdgJ8H',
       'R9p1nT', 'oP45pZ', 'nO7HpJ', 'lUyB55', 'pcz93t', 'f5P44x', 'x9l8DR',
       'FxYUXs', 'LPGxK6', 'zqN0I9', 'DdrBNO', 'i5H0bk', 'RwBmJ4', 'YRyBYh',
       'qo3Kr0', 'enws7H', 'JX6zal', 'LUedkM', 'DPxNer', 'sBF8OI', 'fHUNzp',
       'pwnDwc', 'QAnf1l', 'KPbhd6'],
      dtype='object', name='id')

In [6]:
forms=forms.loc[['fHUNzp']] #, 'pwnDwc', 'QAnf1l', 'KPbhd6']]

In [7]:
field  = {}
fields = []

formItemsColumns=['id','form','type','title']

for form in forms.index:
    r=requests.get(formItems.format(id=form),
           headers=header).json()
    if 'fields' in r:
        for f in r['fields']:
            field = {}
            field['form'] = form
            field['id'] = f['id']
            field['title'] = f['title']
            field['type'] = f['type']
            if 'description' in f:
                field['description'] = f['description']

            fields.append(field)

    if 'hidden' in r:
        for f in r['hidden']:
            idCalc=hashlib.new('shake_256')

            field = {}
            field['form'] = form
            field['title'] = f
            field['type'] = 'hidden'
            
            idCalc.update('{}hidden{}'.format(form,f).encode('utf-8'))
            field['id'] = idCalc.hexdigest(5)

            fields.append(field)

            

formItems=pd.DataFrame(columns=formItemsColumns)
formItems=formItems.append(fields)
formItems.set_index('id',inplace=True)

In [8]:
formItems

,form,type,title
id,,,
47109497,fHUNzp,multiple_choice,Qual o processo que você utilizou?
45269537,fHUNzp,rating,Como foi sua experiência?
45267767,fHUNzp,long_text,Que pena! O que ficou faltando?
45267821,fHUNzp,long_text,Que bom! O que você sugere para que possamos m...
45268023,fHUNzp,opinion_scale,E qual a probabilidade de você continuar utili...
47110010,fHUNzp,short_text,"Se possível, *deixe seu email* para novas pes..."
49215093,fHUNzp,short_text,"Se possível, *deixe seu nome e telefone com DD..."
622eae997e,fHUNzp,hidden,cpfcnpj
cef358a02e,fHUNzp,hidden,cpd


In [9]:
meta     = {}
metas    = []
answer   = {}
answers  = []

metaColumns=['id', 'form_id', 'landed', 'submitted', 'agent', 'referer']
answerColumns=['id', 'form_id', 'response_id', 'field_id', 'answer']


for form in forms.index:
    responses=requests.get(respList.format(id=form,psize=1,page=1),
               headers=header).json()
    for page in range(1,math.ceil(responses['total_items']/1000)):
        responseSet=requests.get(respList.format(id=form,psize=1000,page=page),
                   headers=header).json()

        
        for i in responseSet['items']:
#             print(i)
            meta = {}
            meta['id']          = i['response_id']
            meta['form_id']     = form
            meta['landed']      = i['landed_at']
            meta['submitted']   = i['submitted_at']
            meta['agent']       = i['metadata']['user_agent']
            meta['referer']     = i['metadata']['referer']

            metas.append(meta)

            if 'hidden' in i.keys():
                for field in i['hidden'].keys():
                    idCalc=hashlib.new('shake_256')
                    answer = {}
                    answer['response_id'] =  meta['id']
                    answer['form_id']     =  form
                    idCalc.update('{}{}{}'.format(answer["form_id"],answer["response_id"],field).encode('utf-8'))
                    answer['id']          =  idCalc.hexdigest(5)
                    answer['answer']      =  i['hidden'][field]

                    
                    
                    idCalc=hashlib.new('shake_256')
                    idCalc.update('{}hidden{}'.format(answer["form_id"],field).encode('utf-8'))
                    answer['field_id']    =  idCalc.hexdigest(5)
                    
                    answers.append(answer)
                    

                
            if 'answers' in i.keys():
                for field in i['answers']:
#                     print(f'\t{field}')
                    idCalc=hashlib.new('shake_256')
    
                    answer = {}
                    answer['response_id'] =  meta['id']
                    answer['form_id']     =  form
                    answer['field_id']    =  field['field']['id']
                    idCalc.update('{}{}{}'.format(answer["form_id"],answer["response_id"],answer["field_id"]).encode('utf-8'))
                    answer['id']          =  idCalc.hexdigest(5)


                    if field['type'] == 'choices':
                        answer['answer']=[]
                        for k in field[field['type']].keys():
                            for a in field[field['type']][k]:
                                answer['answer'].append(a)
                        answer['answer']='|'.join(answer['answer'])
                    elif field['type'] == 'choice':
                        for k in field[field['type']].keys():
                            answer['answer']=field[field['type']][k]
                    else:
                        answer['answer'] = field[field['type']]

                    answers.append(answer)


meta=pd.DataFrame(columns=metaColumns)
if len(metas)>0:
    meta=meta.append(metas)
meta.set_index('id',inplace=True)
del metas

answer=pd.DataFrame(columns=answerColumns)
if len(answers)>0:
    answer=answer.append(answers)
answer.set_index('id',inplace=True)
del answers

In [10]:
meta.head()

,form_id,landed,submitted,agent,referer
id,,,,,
9e0d355900651c5d748783ab1a907be6,fHUNzp,2017-06-10T00:04:32Z,2017-06-10T00:06:39Z,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,https://projetosaf.typeform.com/to/fHUNzp?cpfc...
1b4d6acd85ee921c7a7319d3b0f7baaf,fHUNzp,2017-06-09T23:36:20Z,2017-06-09T23:36:56Z,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,https://projetosaf.typeform.com/to/fHUNzp?cpfc...
5c608fccfd84e762168b581a292de5ea,fHUNzp,2017-06-09T21:28:34Z,2017-06-09T21:29:12Z,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,https://projetosaf.typeform.com/to/fHUNzp?cpfc...
06214d3d8b1fffa2aa38627ebd26f25c,fHUNzp,2017-06-09T21:18:45Z,2017-06-09T21:19:36Z,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,https://projetosaf.typeform.com/to/fHUNzp?cpfc...
51ba792028236ecbf4f6f7ef2df3087b,fHUNzp,2017-06-09T19:58:13Z,2017-06-09T19:58:43Z,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,https://projetosaf.typeform.com/to/fHUNzp?cpfc...


In [11]:
answer.head(100)

,form_id,response_id,field_id,answer
id,,,,
0e0c239595,fHUNzp,9e0d355900651c5d748783ab1a907be6,9c55ac84bf,25429640
530d42eb25,fHUNzp,9e0d355900651c5d748783ab1a907be6,cef358a02e,408516
0547af0664,fHUNzp,9e0d355900651c5d748783ab1a907be6,622eae997e,01181334000171
2846b4b212,fHUNzp,9e0d355900651c5d748783ab1a907be6,21d21a0ff4,DUPLICAR_COTACAO
a3fb9960ff,fHUNzp,9e0d355900651c5d748783ab1a907be6,b8ed555823,978
341d5090bf,fHUNzp,9e0d355900651c5d748783ab1a907be6,47109497,Seguro novo / Renovação congênere
e53ba117eb,fHUNzp,9e0d355900651c5d748783ab1a907be6,45269537,3
fe61dd1e11,fHUNzp,9e0d355900651c5d748783ab1a907be6,45267767,O sistema gera um erro e não avisa com clareza...
d5a9c589ef,fHUNzp,9e0d355900651c5d748783ab1a907be6,45268023,5


## Grava o DataFrame para um DB SQLite
Este é o código para abrir ou criar um aquivo SQLite chamado `VivaReal.db`, na tabela `venda`. Para usar outro banco, como MariaDB, Oracle, DB2, altere esta célula e use a biblioteca SQL Alchemy com os drivers corretos. Todo o resto do código funcionará igual.

In [12]:
dbfile = 'TypeForm.db'

db = sqlite3.connect(dbfile)

In [13]:
forms.to_sql('forms', index=True, if_exists='replace', con=db)
formItems.to_sql('formitems', index=True, if_exists='replace', con=db)
meta.to_sql('responses', index=True, if_exists='replace', con=db)
answer.to_sql('responseitems', index=True, if_exists='replace', con=db)

In [14]:
db.commit()
db.close()